In [1]:
import collections
import pre
import numpy as np
import project_tests as tests
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

Using TensorFlow backend.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5352539906871934114
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3707174912
locality {
  bus_id: 1
}
incarnation: 306580462313494756
physical_device_desc: "device: 0, name: GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [2]:
# Load English data
english_sentences = pre.load_data('data/small_vocab_en')
# Load French data
french_sentences = pre.load_data('data/small_vocab_fr')

print('Dataset Loaded')


Dataset Loaded


In [3]:
for sample_i in range(2):
    print('small_vocab_en Line {}:{}'.format(sample_i + 1,english_sentences[sample_i]))
    print('small_vocab_fr Line {}:{}'.format(sample_i + 1,french_sentences[sample_i]))

small_vocab_en Line 1:new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


In [4]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


In [5]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    x_tk = Tokenizer(char_level = False)
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'my': 12, 'prize': 17, 'brown': 4, 'of': 14, 'study': 13, 'short': 20, 'lexicography': 15, 'is': 19, 'jumps': 6, 'a': 3, 'lazy': 8, 'sentence': 21, 'jove': 11, 'fox': 5, 'over': 7, 'this': 18, 'the': 1, 'dog': 9, 'by': 10, 'quick': 2, 'won': 16}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


In [6]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = length, padding = 'post')

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


In [7]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)


Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


In [8]:
def logit_to_text(logits,tokenizer):
    index_to_words = {id:word for word,id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits,1)])
print("'logits_to_test' function loaded.")

'logits_to_test' function loaded.


In [9]:


def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    learning_rate = 1e-3
    input_seq = Input(input_shape[1:])
    rnn = GRU(64, return_sequences = True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logit_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))


Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 7s - loss: 3.4878 - acc: 0.4142 - val_loss: nan - val_acc: 0.4580
Epoch 2/10
110288/110288 [==============================] - 6s - loss: 2.4089 - acc: 0.4793 - val_loss: nan - val_acc: 0.4994
Epoch 3/10
110288/110288 [==============================] - 6s - loss: 2.1412 - acc: 0.5169 - val_loss: nan - val_acc: 0.5370
Epoch 4/10
110288/110288 [==============================] - 6s - loss: 1.9009 - acc: 0.5593 - val_loss: nan - val_acc: 0.5722
Epoch 5/10
110288/110288 [==============================] - 6s - loss: 1.7478 - acc: 0.5744 - val_loss: nan - val_acc: 0.5794
Epoch 6/10
110288/110288 [==============================] - 6s - loss: 1.6590 - acc: 0.5824 - val_loss: nan - val_acc: 0.5878
Epoch 7/10
110288/110288 [==============================] - 6s - loss: 1.5956 - acc: 0.5909 - val_loss: nan - val_acc: 0.5945
Epoch 8/10
110288/110288 [==============================] - 6s - lo

In [18]:
from keras.models import Sequential
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 1e-3
    rnn = GRU(64, return_sequences=True, activation="tanh")
    
    embedding = Embedding(french_vocab_size, 64, input_length=input_shape[1]) 
    logits = TimeDistributed(Dense(french_vocab_size, activation="softmax"))
    
    model = Sequential()
    #em can only be used in first layer --> Keras Documentation
    model.add(embedding)
    model.add(rnn)
    model.add(logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model
tests.test_embed_model(embed_model)


# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# TODO: Train the neural network

embeded_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

embeded_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)


# TODO: Print prediction(s)
print(logit_to_text(embeded_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 6s - loss: 3.7800 - acc: 0.4017 - val_loss: nan - val_acc: 0.4093
Epoch 2/10
110288/110288 [==============================] - 6s - loss: 2.7124 - acc: 0.4349 - val_loss: nan - val_acc: 0.4934
Epoch 3/10
110288/110288 [==============================] - 6s - loss: 2.0977 - acc: 0.5455 - val_loss: nan - val_acc: 0.5996
Epoch 4/10
110288/110288 [==============================] - 6s - loss: 1.5045 - acc: 0.6419 - val_loss: nan - val_acc: 0.6909
Epoch 5/10
110288/110288 [==============================] - 6s - loss: 1.1385 - acc: 0.7264 - val_loss: nan - val_acc: 0.7494
Epoch 6/10
110288/110288 [==============================] - 6s - loss: 0.9219 - acc: 0.7661 - val_loss: nan - val_acc: 0.7840
Epoch 7/10
110288/110288 [==============================] - 6s - loss: 0.7865 - acc: 0.7947 - val_loss: nan - val_acc: 0.8049
Epoch 8/10
110288/110288 [==============================] - 6s - lo

In [20]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 1e-3
    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences = True, dropout = 0.1), 
                           input_shape = input_shape[1:]))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model
tests.test_bd_model(bd_model)


# TODO: Train and Print prediction(s)
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

bidi_model = bd_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)


bidi_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

# Print prediction(s)
print(logit_to_text(bidi_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/20
110288/110288 [==============================] - 13s - loss: 2.6901 - acc: 0.4972 - val_loss: 1.7725 - val_acc: 0.5717
Epoch 2/20
110288/110288 [==============================] - 12s - loss: 1.6535 - acc: 0.5863 - val_loss: 1.4825 - val_acc: 0.6111
Epoch 3/20
110288/110288 [==============================] - 12s - loss: 1.4516 - acc: 0.6138 - val_loss: 1.3632 - val_acc: 0.6239
Epoch 4/20
110288/110288 [==============================] - 12s - loss: 1.3442 - acc: 0.6318 - val_loss: 1.2982 - val_acc: 0.6322
Epoch 5/20
110288/110288 [==============================] - 12s - loss: 1.2743 - acc: 0.6446 - val_loss: 1.2617 - val_acc: 0.6365
Epoch 6/20
110288/110288 [==============================] - 12s - loss: 1.2229 - acc: 0.6536 - val_loss: 1.2410 - val_acc: 0.6351
Epoch 7/20
110288/110288 [==============================] - 12s - loss: 1.1903 - acc: 0.6580 - val_loss: 1.2382 - val_acc: 0.6344
Epoch 8/20
110288/110288 [=============

In [22]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    learning_rate = 1e-3
    model = Sequential()
    model.add(GRU(128, input_shape = input_shape[1:], return_sequences = False))
    model.add(RepeatVector(output_sequence_length))
    model.add(GRU(128, return_sequences = True))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model
tests.test_encdec_model(encdec_model)


# OPTIONAL: Train and Print prediction(s)
tmp_x = pad(preproc_english_sentences)
tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[1], 1))

encodeco_model = encdec_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

encodeco_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

print(logit_to_text(encodeco_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/20
110288/110288 [==============================] - 11s - loss: 3.0352 - acc: 0.4368 - val_loss: 2.4734 - val_acc: 0.4770
Epoch 2/20
110288/110288 [==============================] - 11s - loss: 2.3250 - acc: 0.4949 - val_loss: 2.2024 - val_acc: 0.5058
Epoch 3/20
110288/110288 [==============================] - 10s - loss: 2.0797 - acc: 0.5131 - val_loss: 1.9654 - val_acc: 0.5297
Epoch 4/20
110288/110288 [==============================] - 10s - loss: 1.8897 - acc: 0.5388 - val_loss: 1.8071 - val_acc: 0.5484
Epoch 5/20
110288/110288 [==============================] - 10s - loss: 1.7386 - acc: 0.5579 - val_loss: 1.6715 - val_acc: 0.5684
Epoch 6/20
110288/110288 [==============================] - 10s - loss: 1.6310 - acc: 0.5766 - val_loss: 1.5843 - val_acc: 0.5822
Epoch 7/20
110288/110288 [==============================] - 11s - loss: 1.5487 - acc: 0.5872 - val_loss: 1.5092 - val_acc: 0.5945
Epoch 8/20
110288/110288 [=============

In [23]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    model = Sequential()
    model.add(Embedding(input_dim=english_vocab_size,output_dim=128,input_length=input_shape[1]))
    model.add(Bidirectional(GRU(256,return_sequences=False)))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(GRU(256,return_sequences=True)))
    model.add(TimeDistributed(Dense(french_vocab_size,activation='softmax')))
    learning_rate = 0.005
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model
tests.test_model_final(model_final)


print('Final Model Loaded')

Final Model Loaded


In [24]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    tmp_X = pad(preproc_english_sentences)
    model = model_final(tmp_X.shape,
                        preproc_french_sentences.shape[1],
                        len(english_tokenizer.word_index)+1,
                        len(french_tokenizer.word_index)+1)
    
    model.fit(tmp_X, preproc_french_sentences, batch_size = 1024, epochs = 17, validation_split = 0.2)
 
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Train on 110288 samples, validate on 27573 samples
Epoch 1/17
110288/110288 [==============================] - 40s - loss: 2.0063 - acc: 0.5358 - val_loss: 1.2976 - val_acc: 0.6532
Epoch 2/17
110288/110288 [==============================] - 39s - loss: 0.9941 - acc: 0.7188 - val_loss: 0.8062 - val_acc: 0.7634
Epoch 3/17
110288/110288 [==============================] - 38s - loss: 0.6363 - acc: 0.8100 - val_loss: 0.4932 - val_acc: 0.8506
Epoch 4/17
110288/110288 [==============================] - 38s - loss: 0.3500 - acc: 0.8979 - val_loss: 0.2589 - val_acc: 0.9273
Epoch 5/17
110288/110288 [==============================] - 39s - loss: 0.2269 - acc: 0.9344 - val_loss: 0.1854 - val_acc: 0.9469
Epoch 6/17
110288/110288 [==============================] - 39s - loss: 0.1543 - acc: 0.9558 - val_loss: 0.1401 - val_acc: 0.9596
Epoch 7/17
110288/110288 [==============================] - 39s - loss: 0.1175 - acc: 0.9659 - val_loss: 0.1153 - val_acc: 0.9668
Epoch 8/17
110288/110288 [=============